<a href="https://colab.research.google.com/github/jlopetegui98/RebornToBeWilde/blob/main/2-FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tuning Mistral-7B (instruction) with Oscar Wilde texts**

In [ ]:
# uncomment the following line to run in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# uncomment the following line to run in colab
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

In [ ]:
# uncomment the following line to run in colab
!nvidia-smi

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, gradio, warnings
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
import json
import utils

In [ ]:
# model to fine-tune
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
dataset_name = "finetuning_instructions_dataset_wilde"

In [ ]:
# path to the data
# root_path = '.' # comment if using colab
root_path = './drive/MyDrive/RebornToBeWilde' # uncomment if using colab
dir_data = f'{root_path}/data/{dataset_name}.json'
models_path = f'{root_path}/models'

In [ ]:
# load the dataset
data_dict = json.load(open(dir_data))
dataset = Dataset.from_dict(data_dict)
dataset = dataset.train_test_split(test_size = 0.1, seed = 42)
dataset

In [ ]:
# example of the dataset
print(dataset['train']['text'][0])

In [ ]:
# Load base model(Mistral 7B-Instruct)
# quantization config
model = load_model(model_name, adapt = True)
# print the model
model

In [ ]:
# Load tokenizer
tokenizer = load_tokenizer(model_name)

In [ ]:
#Hyperparamters for fine tuning
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 2,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    # report_to="wandb"
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# evaluate the model
trainer.evaluate()

In [ ]:
# save fine tuned model
model_save_name = 'Mistral7B_fine_tuned_OscarWilde.pt'
path = f"{models_path}/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
print("Model saved successfully!")

Model saved successfully!
